# Modelo de programación lineal propuesto por Almasarwah et. al

In [ ]:
import pandas as pd
import gurobipy as grb

## Modelo

In [ ]:
def main():
    
    #Entradas:Colección de trabajos A[0…ith], mediante una tabla con el formato "JOB,PT,Deadline" en la primera fila donde la columna JOB provee el indicador del trabajo, PT el tiempo de procesamiento y Deadline 
    #         plazo maximo de entrega del trabajo i, tal que 0≤i≤ith.
    #Salidas:Máxima cantidad de trabajos de A[0…ith] que se pueden elaborar sin violar su tiempo límite de entrega
    
    datafile=input("Ingrese el nombre del archivo donde posee las solicitudes a agendar acompañado del tipo de archivo (.csv o .txt)\n")
    data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

    data["JOB"]="J"+data["JOB"].map(str)
    dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

    #Los trabajos son ordenados de acuerdo con la regla EDD (Plazo de entrega más temprano), tal que Deadline0≤…Deadlinei…≤DeadlineI,
    #JOB0≤…JOBi…≤JOBI y son añadidos dentro de un diccionario, tomando como clave su indice JOBi y valor la tupla (JOBi,Deadlinei,PTi).        
    
    dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
    ordDict={}
    cont=0
    for i in dictJobs:
        ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
        cont+=1

    #Se crea una lista para alojar los tiempos de procesamiento p[n] y plazos de entrega d[n].
    p = []
    d = []
    for i in ordDict.keys():
        p.append(ordDict[i][2])
        d.append(ordDict[i][1])

    #Conjuntos:
    #Se crea el conjunto N y M, tal que N={0,..,i,..,ith} y M={0,..,j,..,jth} donde ith hace referencia al último trabajo del 
    #conjunto y jth a la última máquina del conjunto.
    ith = len(ordDict)
    jth = 2
    N = range(ith)
    M = range(jth)
    indices=[(i,j,k) for i in N for j in M for k in N]

    #Parametros:
    #Se crea el parametro t[n] hace referencia al tiempo de finalización de la jornada anterior y WT a las horas disponiples en
    #las que un operario pueda procesar trabajos. Vale recalcar que los tiempos de finalización no toman en cuenta los fines de
    #semanas, dado que en estos periodos el operario no puede acudir a las instalaciones.
    t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
    WT = 11

    #Inicializar modelo
    mdl = grb.Model("modelo")

    #Variables:
    #x: 1 si el trabajo i esta a tiempo en la máquina j durante el horario disponible del dia k; 0 de lo contrario.
    #MAX: 1 si un trabajo i se agenda en la máquina j el dia k; 0 de lo contrario.
    #x = mdl.addVars(indices, vtype=grb.BINARY)
    x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
    MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

    #Función objetivo: Maximizar el número de trabajos a tiempo.
    mdl.setObjective(sum(x[(i,j,k)] for i in N for j in M for k in N))

    #Restricciones:
    #R1:  Restringe a un trabajo de ser asignado a una máquina y un día.
    mdl.addConstrs((sum(x[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

    #R2:  Restringe a los trabajos asignados dentro del intervalo de atención a no sobrepasar dicha magnitud
    mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

    #R3:  Asegura que los trabajos sean entregados de acuerdo con su plazo de entrega, tal que si algún trabajo i perteneciente
    #     a A[1..it] ha sido asignado a la máquina j del batch k, el nuevo trabajo se verá afectado por el tiempo de finalización
    #     del intervalo de trabajo anterior t[k].
    for i in N:
        for j in M:
            for k in range(i+1):      
                mdl.addConstr(sum((x[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])

    #R4:  Otorga valor a la variable MAX con relación a los i+1 trabajos evaluados, tal que MAX tomará valor de 1 si algún trabajo
    #     dentro de esta rango a sido asignado en la máquina j el dia k.
    for i in N:
        for j in M:
            for k in range(i+1):
                for z in range(i+1):
                    mdl.addConstr(x[(z,j,k)] <=MAX[(i,j,k)])

    mdl.ModelSense = grb.GRB.MAXIMIZE
    mdl.optimize()
    
main()